In [1]:
import numpy as np
import random

# Parameters Config

In [2]:
SUB      = 0
SUB_ADD  = 1
MULTIPLY = 2

GEN_TYPE = SUB

TRAINING_SIZE = 80000
DIGITS = 3
ANS_DIGITS = {
    SUB: DIGITS + 1,
    SUB_ADD: DIGITS + 1,
    MULTIPLY: 2 * DIGITS
}.get(GEN_TYPE, DIGITS + 1)
MAXLEN = DIGITS + 1 + DIGITS
chars = {
    SUB: '0123456789- ',
    SUB_ADD: '0123456789+- ',
    MULTIPLY: '0123456789* '
}.get(GEN_TYPE, '0123456789+-* ')

# Data Generation

In [3]:
questions = []
expected = []
seen = set()
operator = {
    SUB: ['-'],
    SUB_ADD: ['-', '+'],
    MULTIPLY: ['*']
}
ans_switcher = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b
}
ops = operator.get(GEN_TYPE, [None])
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: random.choice(range(10 ** random.choice(range(1, DIGITS + 1))))
    g = lambda: random.choice(ops)
    a, b, op = f(), f(), g()
    if op == '-':
        a, b = sorted((a, b), reverse=True)
    key = tuple((a, b, op))
    if key in seen:
        continue
    seen.add(key)
    query = '{}{}{}'.format(a, op, b).ljust(MAXLEN)
    ans_funct = ans_switcher.get(op, lambda a, b: float('NAN'))
    ans = str(ans_funct(a, b)).ljust(ANS_DIGITS)
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [4]:
for i in range(10):
    print(questions[i], '=', expected[i])

74-10   = 64  
72-6    = 66  
314-5   = 309 
683-67  = 616 
99-74   = 25  
77-2    = 75  
369-27  = 342 
262-9   = 253 
7-3     = 4   
990-43  = 947 


# Processing
- Size of training data:   64,000
- Size of validation data: 16,000

In [5]:
class CharacterTable:
    def __init__(self, chars):
        self.chars  = list(chars)
        self.len    = len(chars)
        self.encode = {}
        for i, key in enumerate(self.chars):
            self.encode[key] = np.zeros(self.len, np.float32)
            self.encode[key][i] = 1.
            
    def encoder(self, C):
        result = np.zeros((len(C), self.len))
        for i, c in enumerate(C):
            try:
                result[i] = self.encode[c]
            except:
                pass
        return result
            
    def decoder(self, x):
        x = x.argmax(axis=-1)
        return ''.join(self.chars[i] for i in x)

In [6]:
ct = CharacterTable(chars)

x = np.zeros((len(questions), MAXLEN, len(chars)), np.float32)
y = np.zeros((len(expected), ANS_DIGITS, len(chars)), np.float32)
for i, sentence in enumerate(questions):
    x[i] = ct.encoder(sentence)
for i, sentence in enumerate(expected):
    y[i] = ct.encoder(sentence)

train_x = x[:18000]
train_y = y[:18000]

validation_x = x[18000:20000]
validation_y = y[18000:20000]

# Model

![model](img/seq2seq.png)

- Using sequence to sequence model
- Encoder: bi-directional LSTM
- Decoder: LSTM

In [7]:
import keras as K
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import Input, LSTM, TimeDistributed, Dropout, RepeatVector, Reshape, Bidirectional, Concatenate, Embedding
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

HIDDEN_SIZE = 256
EMBEDDING_SIZE = 256

model = Sequential()

encoder_inputs = Input(shape=(MAXLEN, len(chars)))
# encoder_inputs = Input(shape=(MAXLEN,))
# embedding = Embedding(len(chars), EMBEDDING_SIZE)(encoder_inputs)
encoder = Bidirectional(LSTM(HIDDEN_SIZE, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
states = [state_h, state_c]

# decoder_inputs = RepeatVector(ANS_DIGITS)(encoder_outputs)
# decoder_outputs = Bidirectional(LSTM(int(HIDDEN_SIZE / 2), return_sequences=True))(decoder_inputs)
# decoder_outputs = TimeDistributed(Dense(len(chars), activation='softmax'))(decoder_outputs)

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Reshape((1, HIDDEN_SIZE * 2))
decoder_lstm = LSTM(HIDDEN_SIZE * 2, return_state=True)

all_outputs = []
inputs = decoder_inputs(encoder_outputs)

first_decoder = True
for _ in range(ANS_DIGITS):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    states = [state_h, state_c]
#     inputs = decoder_inputs(outputs)
    
    # Store the current prediction (we will concatenate all predictions later)
    outputs = Dense(len(chars), activation='softmax')(outputs)
    all_outputs.append(outputs)
    

# Concatenate all predictions
decoder_outputs = Concatenate()(all_outputs)
decoder_outputs = Reshape((ANS_DIGITS, len(chars)))(decoder_outputs)
decoder_outputs = Lambda(lambda x: x[:,::-1])(decoder_outputs)

# Define and compile model as previously
model = Model(encoder_inputs, decoder_outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = int(len(train_x) / 128 / 100) * 100

if batch_size == 0:
    batch_size = 100

    
model.fit(train_x, train_y, 
          batch_size=batch_size, epochs=100, 
          verbose=1, validation_data=[validation_x, validation_y])

# model.fit(np.argmax(train_x, axis=2), train_y, 
#           batch_size=batch_size, epochs=100, 
#           verbose=1, validation_data=[np.argmax(validation_x, axis=2), validation_y])

/home/ddnn_user02/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 12)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 550912      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 512)       0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

18000/18000 [==============================] - 9s 497us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0073 - val_acc: 0.9981
Epoch 27/100
18000/18000 [==============================] - 9s 498us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0076 - val_acc: 0.9980
Epoch 28/100
18000/18000 [==============================] - 9s 491us/step - loss: 9.6989e-04 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9980
Epoch 29/100
18000/18000 [==============================] - 9s 487us/step - loss: 8.4489e-04 - acc: 1.0000 - val_loss: 0.0070 - val_acc: 0.9980
Epoch 30/100
18000/18000 [==============================] - 9s 490us/step - loss: 7.3511e-04 - acc: 1.0000 - val_loss: 0.0062 - val_acc: 0.9983
Epoch 31/100
18000/18000 [==============================] - 9s 480us/step - loss: 6.4394e-04 - acc: 1.0000 - val_loss: 0.0060 - val_acc: 0.9985
Epoch 32/100
18000/18000 [==============================] - 9s 496us/step - loss: 5.6451e-04 - acc: 1.0000 - val_loss: 0.0061 - val_acc: 0.9983
Epoch 33/100


18000/18000 [==============================] - 9s 496us/step - loss: 3.5260e-04 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 0.9990
Epoch 84/100
18000/18000 [==============================] - 9s 496us/step - loss: 3.1044e-04 - acc: 1.0000 - val_loss: 0.0066 - val_acc: 0.9989
Epoch 85/100
18000/18000 [==============================] - 9s 492us/step - loss: 2.7387e-04 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9990
Epoch 86/100
18000/18000 [==============================] - 9s 497us/step - loss: 2.4276e-04 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9989
Epoch 87/100
18000/18000 [==============================] - 9s 496us/step - loss: 2.1528e-04 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9989
Epoch 88/100
18000/18000 [==============================] - 9s 487us/step - loss: 1.9173e-04 - acc: 1.0000 - val_loss: 0.0059 - val_acc: 0.9988
Epoch 89/100
18000/18000 [==============================] - 9s 484us/step - loss: 1.7109e-04 - acc: 1.0000 - val_loss: 0.0063 - val_acc: 0.9988
Epoch

# Error check

In [8]:
print('Error check')
pred = model.predict(x)
err = []
for i in range(len(x)):
    if ct.decoder(pred[i]) != ct.decoder(y[i]):
        err.append(i)
print(len(err), '/', len(x))
print()
print('Prediction'.ljust(MAXLEN + ANS_DIGITS + 3),'Ans')
print('-' * (MAXLEN + ANS_DIGITS * 2 + 4))
for i in err:
    print(ct.decoder(x[i]), '=', ct.decoder(pred[i]), ct.decoder(y[i]))


Error check
309 / 80000

Prediction     Ans
-------------------
190-116 = 84   74  
870-856 = 24   14  
100-29  = 81   71  
503-494 = 19   9   
167-165 = 1    2   
803-794 = 19   9   
288-189 = 199  99  
253-250 = 1    3   
991-101 = 880  890 
209-100 = 909  109 
936-736 = 100  200 
660-659 = 11   1   
493-401 = 82   92  
460-389 = 72   71  
904-898 = 5    6   
600-596 = 10   4   
998-309 = 699  689 
179-130 = 48   49  
277-179 = 998  98  
984-300 = 584  684 
178-174 = 1    4   
990-978 = 2    12  
916-716 = 100  200 
105-98  = 8    7   
931-829 = 10   102 
919-895 = 25   24  
728-727 = 2    1   
100-91  = 8    9   
192-120 = 82   72  
620-613 = 8    7   
169-130 = 38   39  
920-419 = 401  501 
649-610 = 38   39  
101-92  = 89   9   
129-110 = 18   19  
845-838 = 8    7   
669-650 = 1    19  
104-98  = 5    6   
929-832 = 987  97  
701-509 = 182  192 
310-269 = 42   41  
179-174 = 15   5   
109-60  = 48   49  
840-759 = 82   81  
111-106 = 6    5   
913-906 = 8    7   
168-167 = 11   1

# Test

In [9]:
q = '20*0'

q_padding = q.ljust(MAXLEN)[:MAXLEN]
test_x = ct.encoder(q_padding)
pred_y = model.predict(test_x.reshape(-1, MAXLEN, len(chars)))
print(q, '=', ct.decoder(pred_y[0]))


20*0 = 19  


In [10]:
str(float('NAN'))

'nan'